In [29]:
import logging
import pickle
import os
import sys
from typing import Dict
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from MDRMF.models.modeller import Modeller
from MDRMF.dataset import Dataset

class RFModeller(Modeller):

    def __init__(
        self, 
        dataset,
        evaluator=None, 
        iterations=10, 
        initial_sample_size=10, 
        acquisition_size=10, 
        acquisition_method="greedy", 
        retrain=True,
        seeds=[],
        feature_importance_opt=None,
        **kwargs) -> None:

        super().__init__(
            dataset, 
            evaluator,
            iterations, 
            initial_sample_size, 
            acquisition_size, 
            acquisition_method, 
            retrain,
            seeds
            )

        self.kwargs = kwargs
        self.model = RandomForestRegressor(**self.kwargs)
        self.feature_importance_opt = feature_importance_opt

        if self.feature_importance_opt is not None:
            self.optimize_for_feature_importance(self.feature_importance_opt)

    def fit(self, iterations_in=None):

        if iterations_in is not None:
            feat_opt = True
        else:
            feat_opt = False

        # Seed handling
        if self.seeds is None or len(self.seeds) == 0:
            initial_pts = self._initial_sampler(initial_sample_size=self.initial_sample_size)
        elif isinstance(self.seeds, (list, np.ndarray)) and all(isinstance(i, int) for i in self.seeds):
            self.seeds = list(self.seeds)  # Ensure seeds is a list
            initial_pts = self.dataset.get_points(self.seeds, remove_points=True)
        else:
            logging.error("Invalid seeds. Must be a list or ndarray of integers, or None.")
            return

        if not feat_opt:
            print(f"y values of starting points {initial_pts.y}")
            
        self.model.fit(initial_pts.X, initial_pts.y)
        
        # First evaluation, using only the initial points
        if self.evaluator is not None and feat_opt is False:
            self.call_evaluator(i=-1, model_dataset=initial_pts) # -1 because ´call_evaluator´ starts at 1, and this iteration should be 0.

        # implemented to allow the ´fit´ method to be used internally in the class to support ´feature_importance_opt´.
        if iterations_in is None:
            iterations = self.iterations
        else:
            iterations = iterations_in

        for i in range(iterations):
        # Acquire new points
            acquired_pts = self._acquisition(self.model)

            # Merge old and new points
            if i == 0:
                model_dataset = self.dataset.merge_datasets([initial_pts, acquired_pts])
            else:
                model_dataset = self.dataset.merge_datasets([model_dataset, acquired_pts])

            if self.retrain:
                # Reset model and train
                self.model = RandomForestRegressor(**self.kwargs)
                self.model.fit(model_dataset.X, model_dataset.y)
            else:
                # Train on existing model
                self.model.fit(model_dataset.X, model_dataset.y)

            if self.evaluator is not None and feat_opt is False:
                self.call_evaluator(i=i, model_dataset=model_dataset)

            if feat_opt:
                self._print_progress_bar(iteration=i, total=iterations)

        if feat_opt:
            print("\n")

        return self.model
    

    def predict(self, dataset: Dataset):

        if isinstance(dataset, Dataset):
            return self.model.predict(dataset.X)
        else:
            logging.error("Wrong object type. Must be of type `Dataset`")


    def save(self, filename: str):
        """
        Save the RFModeller to a pickle file
        """
        # Check if filename is a string.
        if not isinstance(filename, str):
            raise ValueError("filename must be a string")
        
        try:
            with open(filename, "wb") as f:
                pickle.dump(self, f)
        except FileNotFoundError:
            logging.error(f"File not found: {filename}")
            raise
        except IOError as e:
            logging.error(f"IOError: {str(e)}")
            raise
        except pickle.PicklingError as e:
            logging.error(f"Failed to pickle model: {str(e)}")
            raise
        except Exception as e:
            logging.error(f"Unexpected error: {str(e)}")
            raise


    @staticmethod
    def load(filename: str):
        
        # Check if filename is a string.
        if not isinstance(filename, str):
            raise ValueError("filename must be a string")
        
        # Check if file exists.
        if not os.path.isfile(filename):
            raise FileNotFoundError(f"No such file or directory: '{filename}'")
        
        try:
            with open(filename, "rb") as f:
                return pickle.load(f)
        except FileNotFoundError:
            logging.error(f"File not found: {filename}")
            raise
        except IOError as e:
            logging.error(f"IOError: {str(e)}")
            raise
        except pickle.UnpicklingError as e:
            logging.error(f"Failed to unpickle model: {str(e)}")
            raise
        except Exception as e:
            logging.error(f"Unexpected error: {str(e)}")
            raise


    def optimize_for_feature_importance(self, opt_parameters: Dict):

        print('Computing feature importance...')

        iterations = opt_parameters['opt_iterations']
        features_limit = opt_parameters['opt_features_limit']
    
        model = self.fit(iterations_in=iterations)

        feature_importances = model.feature_importances_
        feature_importances_sorted = np.argsort(feature_importances)[:-1]
        important_features = feature_importances_sorted[-features_limit:]

        self.dataset.X = self.dataset.X[:, important_features]
        self.eval_dataset.X = self.eval_dataset.X[:, important_features]

        # important_feature_values = feature_importances[important_features]
        # print(f"values of most important features: {important_feature_values}")
        
        print(f"Indices of most important features: {important_features} \n")
   

        return important_features

        # --- Comments
        # This function is not yet implemented.
        # There should be an argument to RFModeller called ´feature_importance_opt´.
        # This argument should take a dict.
        # dict = {
        #   'opt_iterations': 50
        #   'opt_features_limit': 30
        # }
        # This list contain the number of times to train the optimization model and how many of the most important features to keep.
        # For each run the ´feature_importances_´ is calculated. The index values are counted. So for instance
        # 1. run index 55 is the most important feature
        # 2. run index 55 is the 10th most important feature
        # 3. run index 55 is the 5th most important feature
        # Now we just average how much 55 was used. (1+10+5)/3 = 5.33
        # We then just calculate an average for each index in the vector and sort them from lowest(best) to highest(worst).
        # In the case of the above dict we only keep the 30 most important features in the vector.
        # -----
        # Now that I think about it we might not even need to do this averaging, as the numbers are kind of already
        # averaged by merely running the model many times.
        # -----
        # Once the desired features have been found we need to set he dataset.X to the indexes that was found most important.
        # I think we can do this by just manipulating self.dataset, but I am a little unsure if this will disturb other parts
        # of the code. I don't think so, as we never return the Dataset at any time.

    def _print_progress_bar(self, iteration, total, bar_length=50, prefix="Progress"):
        """
        Print the progress bar.

        Args:
            iteration (int): current iteration.
            total (int): total iterations.
            bar_length (int): length of the progress bar.
            prefix (str): Prefix to print before the progress bar. Default is "Progress".
        """
        iteration = iteration + 1
        progress = (iteration / total)
        arrow = '-' * int(round(progress * bar_length) - 1) + '>'
        spaces = ' ' * (bar_length - len(arrow))

        sys.stdout.write(f"\r{prefix}: [{arrow + spaces}] {int(progress * 100)}% ({iteration}/{total})")
        sys.stdout.flush()
     

In [2]:
from MDRMF import MoleculeLoader

data = MoleculeLoader('10K.csv', 'SMILES', 'r_i_docking_score').df
data = data.iloc[:-5000]

In [3]:
from MDRMF import Featurizer
feat = Featurizer(data)
#features = feat.featurize("morgan", radius=2, nBits=1024)
features = feat.featurize("rdkit2D")

Computing features...
Progress: [------------------------------------------------->] 100% (4898/4898)
Feature computation completed.


In [4]:
X = features
y = data['r_i_docking_score']
ids = data['SMILES']

dataset = Dataset(X=X, y=y, ids=ids)

In [5]:
dataset.X

array([[13.29054352, -0.1670179 , 13.29054352, ...,  0.        ,
         0.        ,  2.        ],
       [13.22061492, -0.56435377, 13.22061492, ...,  0.        ,
         0.        ,  0.        ],
       [13.34317359, -0.13321653, 13.34317359, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [12.8514159 , -0.24114315, 12.8514159 , ...,  0.        ,
         0.        ,  0.        ],
       [13.12679538, -2.19126387, 13.12679538, ...,  0.        ,
         0.        ,  0.        ],
       [12.51110386, -4.49435943, 12.51110386, ...,  0.        ,
         0.        ,  0.        ]])

In [6]:
# unimportant_features = np.array([1, 4, 7, 9])

# for mol in dataset.X:

#     mol_important_features = np.delete(mol, unimportant_features)

#     print(mol_important_features, end='\n\n')

In [7]:
from MDRMF import Evaluator
metrics = ['top-k', 'top-k-acquired']
k_values = ['100']
eval = Evaluator(dataset, metrics, k_values)

In [41]:
model = RFModeller(dataset, eval, 10, feature_importance_opt={'opt_iterations': 5, 'opt_features_limit': 120})
model.fit()
#model.optimize_for_feature_importance({'opt_iterations': 5, 'opt_features_limit': 2})

208
Computing feature importance...
Progress: [------------------------------------------------->] 100% (5/5)

Indices of most important features: [ 50 138 153 152 102 105 103 119 104 133 134 113  47  77 143  66 109 108
 199 164  25  65   8  35  82 106  52  38  49  89 165 128  87  93 100  12
  70  27 116  21 115  22  81  45   5  99  69  36  59 192  90  85 139 120
  73  83  48  57  41  74  75  15  29   7  92  63  56 112  14  51   3  16
  34 207  95  55 159  11   2  17  68  88   0  46   1  42  19  32  79  43
  13  37  91 150 122  60  98  39  23 187  40  58  97  10 101  33   4  62
  96  18  64  76  86  30  44  28  20  24 121  31] 

y values of starting points [-8.99928 -8.04701 -8.58527 -8.85926 -9.41292 -8.97073 -9.57385 -8.35605
 -9.80547 -8.60594]
Iteration 0, Results: {'top-100 model': 0.05, 'top-100 acquired': 0.0}
Iteration 1, Results: {'top-100 model': 0.01, 'top-100 acquired': 0.0}
Iteration 2, Results: {'top-100 model': 0.05, 'top-100 acquired': 0.0}
Iteration 3, Results: {'top-1

RandomForestRegressor()

In [9]:
model = RFModeller(dataset, eval, 10)
model.fit()

208
y values of starting points [ -9.55452 -10.2797   -9.21848  -8.99632 -10.3181   -9.72285  -8.93664
  -8.84411  -8.46922  -8.2767 ]
Iteration 0, Results: {'top-100 model': 0.18, 'top-100 acquired': 0.0}
Iteration 1, Results: {'top-100 model': 0.21, 'top-100 acquired': 0.01}
Iteration 2, Results: {'top-100 model': 0.28, 'top-100 acquired': 0.06}
Iteration 3, Results: {'top-100 model': 0.27, 'top-100 acquired': 0.13}
Iteration 4, Results: {'top-100 model': 0.3, 'top-100 acquired': 0.16}
Iteration 5, Results: {'top-100 model': 0.29, 'top-100 acquired': 0.18}
Iteration 6, Results: {'top-100 model': 0.28, 'top-100 acquired': 0.19}
Iteration 7, Results: {'top-100 model': 0.32, 'top-100 acquired': 0.2}
Iteration 8, Results: {'top-100 model': 0.3, 'top-100 acquired': 0.23}
Iteration 9, Results: {'top-100 model': 0.29, 'top-100 acquired': 0.23}
Iteration 10, Results: {'top-100 model': 0.33, 'top-100 acquired': 0.23}


RandomForestRegressor()

In [10]:
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors, MolFromSmiles

# Create RDKit molecule object from SMILES string
mol = MolFromSmiles('O=C(C)Oc1ccccc1C(=O)O')

# Check if the molecule is valid
if mol:
    rdkit2D_descriptors = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors.descList])
    descriptor_names = rdkit2D_descriptors.GetDescriptorNames()
    descriptor_values = rdkit2D_descriptors.CalcDescriptors(mol)
    
    for name, value in zip(descriptor_names, descriptor_values):
        print(f"{name}: {value}")
else:
    print("Invalid molecule")

MaxEStateIndex: 10.611948223733938
MinEStateIndex: -1.1140277777777776
MaxAbsEStateIndex: 10.611948223733938
MinAbsEStateIndex: 0.01601851851851821
qed: 0.5501217966938848
MolWt: 180.15899999999996
HeavyAtomMolWt: 172.09499999999997
ExactMolWt: 180.042258736
NumValenceElectrons: 68
NumRadicalElectrons: 0
MaxPartialCharge: 0.33900378687731025
MinPartialCharge: -0.4775395271554559
MaxAbsPartialCharge: 0.4775395271554559
MinAbsPartialCharge: 0.33900378687731025
FpDensityMorgan1: 1.3076923076923077
FpDensityMorgan2: 1.9230769230769231
FpDensityMorgan3: 2.4615384615384617
BCUT2D_MWHI: 16.53611122125436
BCUT2D_MWLOW: 10.182282381035359
BCUT2D_CHGHI: 2.104306980957853
BCUT2D_CHGLO: -2.0311320919470175
BCUT2D_LOGPHI: 2.169834556812799
BCUT2D_LOGPLOW: -2.063000061964294
BCUT2D_MRHI: 5.913065796110137
BCUT2D_MRLOW: -0.13141434244030484
BalabanJ: 3.0435273546341013
BertzCT: 343.2228677267164
Chi0: 9.844934982691242
Chi0n: 6.9813595436500515
Chi0v: 6.9813595436500515
Chi1: 6.109060905280622
Chi1n: